# Integrated Slew

<p style="color: red"> Since this notebook is so close to the one required to run LVV-T2216, I will re-run it using the slew command. This will tell us if we can slew and track while have the mirrors LUTs using the mount telemetry and what are the limitations. </p>

This notebook is used to execute the [LVV-T2215 (2.0)] test script during System Spread Integration Tests on Level 3.  
It is part of the plan [LVV-P81] and of the test cylce [LVV-C175].   
Execution steps are separated by horizontal lines.   
Upon completion, save the notebook and its output as a pdf file to be attached to the test execution in JIRA.

History:
 - [LVV-T2215 (1.0)] has a notebook which doesn't do the slew sequence described on the test script. For this reason the version 2.0 was created.

 - [LVV-T2215 (2.0)] is similar to [LVV-T2290 (2.0)] test case, but without taking a ComCam image. 
 
Execution steps are separated by horizontal lines. Upon completion, save the notebook and its output as a pdf file to be attached to the test execution in JIRA. 

Last executed by J. Esteves 20220408

[LVV-T2215 (1.0)]: https://jira.lsstcorp.org/secure/Tests.jspa#/testCase/2590
[LVV-T2215 (2.0)]: https://jira.lsstcorp.org/secure/Tests.jspa#/testCase/LVV-T2215
[LVV-T2290 (2.0)]: https://jira.lsstcorp.org/secure/Tests.jspa#/testCase/LVV-T2290  
[LVV-P81]: https://jira.lsstcorp.org/secure/Tests.jspa#/testPlan/LVV-P81
[LVV-C175]: https://jira.lsstcorp.org/secure/Tests.jspa#/testCycle/LVV-C175

---
## Under Construction:


Things to do:
1. Add log info
2. Test on Tucson 


In [ ]:
from lsst.ts import utils

# Extract your name from the Jupyter Hub
__executed_by__ = os.environ["JUPYTERHUB_USER"]  

# Extract execution date
__executed_on__ = utils.astropy_time_from_tai_unix(utils.current_tai())
__executed_on__.format = "isot"

# This is used later to define where Butler stores the images
summit = os.environ["LSST_DDS_PARTITION_PREFIX"] == "summit"

print(f"\nExecuted by {__executed_by__} on {__executed_on__}."
      f"\n  At the summit? {summit}")

---
## Load all the needed libraries

Using the setup procedure, get the remotes and the components ready.

This includes simulators as well as real hardware when available (this will depend on when the test is conducted at NCSA or on level 3 or on the telescope):

- pointing  
- mount ( with the CCW)  
- rotator  
- ready M1M3: raise mirror, turn on FB, clear forces. Note that if used at level 3, we need to have M1M3 LUT use mount telemetry  
- ready M2: turn on FB, clear forces. Note that if used at level 3, we need to have M2 LUT use mount telemetry  
- Get cam hex Ready: check config; make sure LUT is on and has valid inputs; make sure hex is at LUT position  
- Get M2 hex (simulator) Ready: check config; make sure LUT is on and has valid inputs; make sure hex is at LUT position  
- Finally, get the MTAOS CSC ready  

***
Run the setup.ipnyb notebook to bring all components up and in their enabled position. Check Chronograph.

***

Check Chronograph.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import rubin_jupyter_utils.lab.notebook as nb
nb.utils.get_node()

In [ ]:
import os
import sys
import asyncio
import logging
import time

import pandas as pd
import numpy as np

from matplotlib import pyplot as plt
from astropy.time import Time
from datetime import datetime, timedelta
import pandas as pd

from lsst.ts import salobj
from lsst.ts.observatory.control.maintel import MTCS, ComCam
from lsst.ts.observatory.control import RotType

In [ ]:
logging.basicConfig(format="%(name)s:%(message)s", level=logging.DEBUG)

In [ ]:
log = logging.getLogger("setup")
log.level = logging.DEBUG

In [ ]:
domain = salobj.Domain()

In [ ]:
mtcs = MTCS(domain=domain, log=log)
mtcs.set_rem_loglevel(10)

In [ ]:
await mtcs.start_task

In [ ]:
async def moveMountInElevationSteps(target_el, azimuth=0, step_size=0.25, time_sleep=1):
    """Move the mount from the current elevation angle to the target elevation angle 
    in steps to avoid any issues whe M1M3 and/or M2 are running with the LUT using the 
    Mount instead of the inclinometer.
    
    This function will actually calculate the number of steps using the ceiling
    in order to make sure that we move carefully. 
    
    Parameters
    ----------
    target_el : float
        Target elevation angle in degrees
    azimuth : float
        Azimuth angle in degres (default)
    step_size : float
        Step elevation size in degrees (default: 0.25)
    time_sleep : float
        Sleep time between movements (default: 1)
        
    Returns
    -------
    azimuth : float
        Current azimuth
    elevation : float
        Current elevation
    """
    current_el = mtcs.rem.mtmount.tel_elevation.get().actualPosition

    if np.abs(current_el - target_el) < step_size:
        print("Too close to move")
        return azimuth, current_el
        
    n_steps = int(np.ceil(np.abs(current_el - target_el) / step_size))

    for el in np.linspace(current_el, target_el, n_steps):
        print(f"Moving elevation to {el:.2f} deg")
        await mtcs.rem.mtmount.cmd_moveToTarget.set_start(azimuth=azimuth, elevation=el)
        time.sleep(time_sleep)
        
    return azimuth, el

---
## Slew Sequence

Exercise the telescope on elevation between 86.5º and 82.5º with 4 slews. <b> No tracking is needed </b>

target_1 -> az = 180.0$^o$, el = 86.5$^o$  
target_2 -> az = 180.0$^o$, el = 85.0$^o$  
target_3 -> az = 180.0$^o$, el = 83.5$^o$  
target_4 -> az = 180.0$^o$, el = 82.0$^o$  

In [ ]:
caution = False

if not caution:
    print("Slew 1: el=86.5")
    await mtcs.point_azel(az=180.0, el=86.5)

    print("Slew 2: el=85.0")
    await mtcs.point_azel(az=180.0, el=85.0)

    print("Slew 3: el=83.5")
    await mtcs.point_azel(az=180.0, el=83.5)

    print("Slew 4: el=82.0")
    await mtcs.point_azel(az=180.0, el=82.0)
    
# await mtcs.stop_tracking()

In [ ]:
await moveMountInElevationSteps(90., azimuth=0)

In [ ]:
await mtcs.set_state(
    state=salobj.State.ENABLED,
    components=["mtm1m3"], 
    overrides = {"mtm1m3": 'Default'}
    )

In [ ]:
await mtcs.raise_m1m3()

In [ ]:
await mtcs.enable_m1m3_balance_system()

In [ ]:
await mtcs.reset_m1m3_forces()

In [ ]:
await moveMountInElevationSteps(85.5, azimuth=190)

***
Slew to target 1:

In [ ]:
await mtcs.stop_tracking()

In [ ]:
target = mtcs.radec_from_azel(az=180., el=85.4)
await mtcs.slew_icrs(ra=target.ra, dec=target.dec, rot_type=RotType.Physical, rot=0)

---
Slew to target_2:

In [ ]:
target = mtcs.radec_from_azel(az=180., el=85.2)
await mtcs.slew_icrs(ra=target.ra, dec=target.dec, rot_type=RotType.Physical, rot=0)

In [ ]:
target = mtcs.radec_from_azel(az=180., el=84.9)
await mtcs.slew_icrs(ra=target.ra, dec=target.dec, rot_type=RotType.Physical, rot=0)

---
Slew to target_3

In [ ]:
await mtcs.point_azel(az=180.0, el=84.5)

---
Slew to target 4

In [ ]:
await mtcs.point_azel(az=180.0, el=84)

In [ ]:
from lsst.ts.observatory.control.maintel import MTCS, ComCam


In [ ]:
comcam = ComCam(domain=domain, log=log)

In [ ]:
comcam.set_rem_loglevel(40)

In [ ]:
await comcam.start_task

In [ ]:
await comcam.standby()

In [ ]:
await comcam.enable()

In [ ]:
exp1 = await comcam.take_object(15)
print(f"Target 1 exposure: {exp1}")

---
Stop tracking to prevent hitting the Rotator soft limit.

In [ ]:
await mtcs.stop_tracking()

***
## Plot The Results


In [ ]:
from lsst_efd_client import EfdClient

In [ ]:
client = EfdClient("summit_efd")

In [ ]:
# Fix this plot
end = Time(datetime.now(), scale='utc')
start = end - timedelta(seconds=1000)

dfm = await client.select_time_series('lsst.sal.MTMount.elevation', 'actualPosition', start, end, 0)
dfm1m3 = await client.select_time_series('lsst.sal.MTM1M3.logevent_appliedElevationForces', 'zForces0', start, end, 0)
dfm2 = await client.select_time_series('lsst.sal.MTM2.axialForces', 'lutGravity0', start, end)
dfh = await client.select_time_series('lsst.sal.MTHexapod.application', '*', start, end)

idx1=dfh.MTHexapodID==1
dfh1 = dfh[idx1]
idx2=dfh.MTHexapodID==2
dfh2 = dfh[idx2]

fig, ax = plt.subplots(1,1, figsize=(15,4))
plt.plot(dfm.actualPosition, '--', label='mount elevation')
plt.plot(dfm1m3.zForces0, label='M1M3 elevation y-force 101')
plt.plot(dfm2.lutGravity0, label='M2 elevation force B1')
plt.plot(dfh1.position1, label='Camera hexapod y')
plt.plot(dfh2.position1, label='M2 hexapod y')
plt.grid()
plt.legend()

In [ ]:
dfm2

***
## Wrap Up and Shut Down

This cell is not currently included as part of the test execution, but included here as needed to shutdown the systems

In [ ]:
await mtcs.set_state(salobj.State.STANDBY, components=["mtaos"])

In [ ]:
await mtcs.lower_m1m3()

In [ ]:
await mtcs.set_state(salobj.State.STANDBY, components=["mtm1m3"])

In [ ]:
await mtcs.set_state(salobj.State.STANDBY, components=["mtm2"])

In [ ]:
await mtcs.set_state(salobj.State.STANDBY, components=["mthexapod_1"])

In [ ]:
await mtcs.set_state(salobj.State.STANDBY, components=["mthexapod_2"])

In [ ]:
await mtcs.standby()